# Imports

In [3]:
import random

# VRPTW

In [4]:
class VRPTW:
    def __init__(self, distance_matrix, time_windows, vehicle_capacity=200, num_vehicles=25, num_ants=25, num_iterations=100):  # Function for initiation with param
        
        # Creating instance variables from params
        self.distance_matrix = distance_matrix
        self.time_windows = time_windows
        self.vehicle_capacity = vehicle_capacity
        self.num_vehicles = num_vehicles
        self.num_ants = num_ants
        self.num_iterations = num_iterations

        self.pheromones = [[1 for _ in range(len(distance_matrix))] for _ in range(len(distance_matrix))]  # Initializing 2D list (matrix); a list of lists
        # Stores the pheromone levels for each route between nodes (customers and the depot); N nodes of distance_matrix results in N x N matrix
        # Rows represent each node
        # Column represents the pheromone level for the route from the row node to the column node
        # for _ in range(len(distance_matrix)), creates outer list with as many rows as there are nodes in the problem (customers + depot); node amount gained from len(distance_matrix)
        # [1 for _ in range(len(distance_matrix))], creates inner list / each row of matrix, initializing each element in row to 1 i.e. all routes are as attractive

        self.best_solution = None

        self.best_cost = float('inf')  # Used to keep track of the best (minimum) cost of a solution found
        # float('inf') is a floating-point value representing positive infinity i.e. the initial value will always be higher than any valid solution later found

    def construct_solution(self):  # Generates potential routes for each ant based on the current state of the problem
        solutions = []  # Initializes storage of potential routes / solutions
        best_cost_this_iteration = float('inf')  # Initialize to a large value for this iteration

        for _ in range(self.num_ants):  # Loop iterates over the number of ants, allowing each ant to construct its own set of routes
            routes = [[] for _ in range(self.num_vehicles)]  # A list of lists, where each inner list represents the route for a specific vehicle
            capacities = [0 for _ in range(self.num_vehicles)]  # A list to keep track of the total load (demand) for each vehicle; initialized to 0
            current_time = [0 for _ in range(self.num_vehicles)]  # A list to track the current time for each vehicle, initialized to 0
            
            # Start from the depot (node 0)
            for vehicle in range(self.num_vehicles):  # Loop iterates over the number of vehicles, allowing each vehicle to build its own route
                current_location = 0  # Depot i.e. start point for all routes
                while True:
                    # Select the next customer to visit
                    next_customer = self.select_next_customer(current_location, capacities[vehicle], current_time[vehicle])  # Calls method responsible for determining the next customer to visit
                    if next_customer is None:  # If there is no viable next customer found by self.select_next_customer e.g. full capacity for vehicle
                        break  # Stop inner loop i.e. vehicle stops its route construction
                    
                    # Update vehicle state i.e. next customer to visit has successfully been selected
                    routes[vehicle].append(next_customer)  # Append the customer to the vehicle route
                    capacities[vehicle] += customer_demand[next_customer]  # Assuming customer_demand is defined, vehicle's capacity is updated by adding the demand of the selected customer
                    
                    # Update current time considering travel time and time windows
                    current_time[vehicle] += self.distance_matrix[current_location][next_customer]  # Travel time to next customer
                    # Ensure the vehicle adheres to time window constraints
                    current_time[vehicle] = max(current_time[vehicle], self.time_windows[next_customer][0])  # Earliest arrival time
                    current_location = next_customer  # Update current_location to the newly visited customer

            solutions.append(routes)  # Adds all the complete set of routes for all ants in the solutions variable initialized at the start of the function

            # Calculate total cost for this solution, including time window penalties
            total_cost = sum(self.calculate_route_cost(route, current_time) for route in routes if route)

            # Check if this is the best solution so far
            if total_cost < best_cost_this_iteration:
                best_cost_this_iteration = total_cost

        return solutions  # Function returns the complete list of routes constructed by all ants when called

    def select_next_customer(self, current_location, current_capacity, current_time):  # Determines which customer an ant should visit next while constructing its route
        probabilities = []  # Initializes storage of which customers can be visited next, along with a score that represents how likely each customer is to be chosen
        for customer in range(1, len(self.distance_matrix)):  # Iterates over all potential customers, except for 0 which is depot
            if current_capacity + customer_demand[customer] <= self.vehicle_capacity:  # Determines if customer can be visited by comparing the vehicles capacity to the current capacity of the route + customer demand
                
                # Calculate pheromone and heuristic values
                pheromone = self.pheromones[current_location][customer]  # Retrieves the pheromone level between the current location and the next customer from pheromone matrix
                distance = self.distance_matrix[current_location][customer]  # Retrieves the travel distance / time from the current location to the next customer from the distance matrix
                heuristic = 1 / (distance + 1e-6)  # Calculate the heuristic value i.e., the inverse of the distance, for later probability calculation; a small constant (1e-6) is added to avoid division by 0
                probability = pheromone * heuristic  # Calculate the probability / attractiveness of the customer by multiplying the pheromone level by the heuristic value
                probabilities.append((customer, probability))  # Adds valid customer, along with its attractiveness, to the probabilities variable initialized at the start, as a tuple
        
        if not probabilities:  # There are no valid customers so function select_next_customer is to return None, a value recognized by construct_solution function
            return None
        
        # Normalize probabilities, making sure the sum becomes 1 i.e. becomes suitable for probabilistic selection
        total = sum(prob for _, prob in probabilities)  # Sums the values of each probability of valid customers

        probabilities = [(customer, prob / total) for customer, prob in probabilities]  # Redefines the list of customers that can be visited with the new probabilities
        
        # Choose the next customer based on probabilities
        return random.choices([customer for customer, _ in probabilities], weights=[prob for _, prob in probabilities])[0]

    def update_pheromones(self, solutions):  # Function for updating of the pheromones on each route; takes in argument solutions which are routes generated by the ants during the current iteration
        # Update pheromone levels based on the quality of solutions
        for solution in solutions:  # Loops over each solution generated by the ants
            total_cost = 0  # Initializes a variable to accumulate the total cost of the routes in the current solution, initialized to 0 as there is no inherent cost
            for route in solution:  # Iterates through each route assigned to a vehicle
                if not route:  # Skips any empty routes i.e. vehicles that didn't deliver to any customers / pick any customers
                    continue  # Goes to next route

                # Calculate cost for the route
                route_cost = self.calculate_route_cost(route)  # Calls function calculate_route_cost (defined further down) to calculate the cost of the current route
                total_cost += route_cost  # Adds the calculated cost of the route to the total cost of the solution
                
                # Update pheromones along the route
                for i in range(len(route) - 1):  # Goes through the segments / paths in the route 
                    self.pheromones[route[i]][route[i + 1]] += 1 / route_cost  # Increase the pheromone level on the path from route[i] to route[i + 1] by the inverse of the route cost
                    # 1 / route_cost is the inverse of the route cost and is used as the increase because the lower the cost, the more pheromone is added, the more desirable the route is for future ants

            # Evaporate pheromones i.e. reduce the pheromone levels across all paths; helps prevent convergence to suboptimal solution and encourages exploration
            for i in range(len(self.pheromones)):  # Iterates over all rows
                for j in range(len(self.pheromones[i])):  # Iterates over all columns in the row
                    self.pheromones[i][j] *= 0.95  # Reduces the pheromone on each path by multiplying with a value below 1

    def calculate_route_cost(self, route, current_time):  # Function to calculate the total cost of a specific route taken by a vehicle; takes argument route which is the route to be evaluated
        cost = 0  # Initialize a variable that accumulates the total distance / time of the route as it iterates through the customers; initialized to 0 as there is no inherent cost
        for i in range(len(route) - 1):  # Iterates over the indices of the route list, stopping before the last customer; stops before the last customer as the cost calculation requires looking at pairs of locations (the current and the next)
            cost += self.distance_matrix[route[i]][route[i + 1]]  # Retrieves the distance / travel time between the current location and the next location, and adds it to the cost variable

        # Add time window penalties
        for i in range(len(route)):
            if current_time[i] < self.time_windows[route[i]][0]:  # Early arrival penalty
                cost += self.time_windows[route[i]][0] - current_time[i]  # Add penalty for arriving before the time window
            elif current_time[i] > self.time_windows[route[i]][1]:  # Late arrival penalty
                cost += current_time[i] - self.time_windows[route[i]][1]  # Add penalty for arriving after the time window

        return cost  # After the for loop has accumulated the total distance / time taken by this route and saved it in cost, the function returns the total cost of the route

    def run(self):  # Function to execute the ACO algorithm over a specified number of iterations
        for _ in range(self.num_iterations):  # Runs for the number of desired iterations
            solutions = self.construct_solution()  # Calls function construct_solution and saves the generated list of routes / solutions for the ants in variable solutions
            
            # Update best solution
            if best_cost_this_iteration < self.best_cost:  # if this iterations best cost is less than the total best cost; less because we want to minimize cost
                self.best_cost = best_cost_this_iteration  # Sets current iterations cost to total best cost
                self.best_solution = solutions  # Sets current solution to total best solution
            
            self.update_pheromones(solutions)  # Calls function update_pheromones, and updates the pheromones on each path using the solutions generated

# c101 constraints

In [ ]:
# Define distance_matrix, time_windows, and customer_demand here
distance_matrix = [
    # Fill with distances between nodes
]
time_windows = [
    # Fill with time windows for each node
]
customer_demand = [
    # Define demands for each customer (index corresponds to customer ID)
]

# Run ACO

In [ ]:
aco = VRPTW(distance_matrix, time_windows, num_vehicles=25, num_ants=25, num_iterations=100)
aco.run()